In [28]:
import numpy as np

def SimplexMatricial(A: np.array, b: np.array, c: np.array, var: np.array, verbose: bool=False, svm: bool=False):
    B = np.eye(len(b))                    # Matriz de bases B de tamaño igual al número de holguras del problema.
    A_original = A.copy()                 # Creamos dos copias de las matrices A, b y c ya que en pasos posteriores se necesitan las matrices originales y vamos a modificarlas.
    b_original = b.copy()
    c_original = -c.copy()
    var_original = var.copy()             # Variables alfabéticas para saber exáctamente qué variable tiene qué valor.
    base = var_original.copy()[-len(b):]  # Base alfabética para saber cuáles son las variables de la base.
    z = 0                                 # Variable en la que guardaremos la solución.
    c_b = np.zeros_like(b)                # Matriz de coeficientes de la base en la función objetivo.

    #if verbose:
      #it = 1
      #print('Base inicial {'+', '.join(base)+'}')

    while np.any(c < 0):                                       # Condición de parada cuando todos los coeficietes de la matriz c son positivos.
        indice_mas_negativo = np.argmin(c)                     # Guardamos la posición del coeficiente de la matrix c más negativo.
        columna_encontrada = A[:, indice_mas_negativo]         # Guardamos la columna de la matriz A que se encuentra debajo del coeficiente c anterior.
        variable_entrada = A_original[:, indice_mas_negativo]  # Guardamos la misma información que en el paso anterior pero de la matriz A original para calcular después la matriz B.
        #if verbose:
          #print(f'Iteración {it}:')
          #print(f'\tFunción objetivo: {list(c)}')
          #print(f'\tVariable de entrada: {var_original[indice_mas_negativo]}')
         # print(f'\tColumna de entrada: {list(columna_encontrada)}')

        idx_aux = [i for i in range(len(columna_encontrada)) if columna_encontrada[i] != 0.0]                     # Si existe algún cero, al quitarlo para que la división no de error es posible que los índices cambien por lo que guardamos dichos índices en una lista.

        division_result = np.divide(b[columna_encontrada != 0.0], columna_encontrada[columna_encontrada != 0.0])  # Realizamos la división de b entre la columna de A, anteriormente guardada, para hallar la variable de salida.

        #if verbose:
         # print('\tMínimo de {'+str(', '.join(list(str(b[i])+"/"+str(columna_encontrada[i]) for i in range(len(b)))))+'} -> ', np.min(division_result[division_result > 0]))

        idx_aux_2 = [i for i in range(len(division_result)) if division_result[i] > 0.0]
        posicion_variable_salida = np.argmin(division_result[division_result > 0.0])      # Guardamos la posición de la variable de salida con la condición de que sea el más pequeño pero positivo.
        posicion_variable_salida = idx_aux_2[posicion_variable_salida]
        posicion_variable_salida = idx_aux[posicion_variable_salida]
        #if verbose:
          #print(f'\tVariable de salida: {base[posicion_variable_salida]}')

        B[:, posicion_variable_salida] = variable_entrada                                 # Modificamos la matriz B, añadiéndole la 'variable_entrada' que es la columna de A original en la posición que hemos obtenido antes que ocupaba la 'variable_salida'.
        base[posicion_variable_salida] = var_original[indice_mas_negativo]

        c_b[posicion_variable_salida] = c_original[indice_mas_negativo]                   # Calculamos 'c_b'.




        order = [(list(var_original).index(base[i]), i) for i in range(len(base))]        # Ordenamos la base manteniendo el orden inicial de las variables.
        correct_order = sorted(order)
        # print(correct_order)
        B_aux = np.zeros_like(B)
        c_b_aux = np.zeros_like(c_b)
        base_aux = np.zeros_like(base)
        for i,base_var in enumerate(correct_order):
          B_aux[:,i] = B[:,base_var[1]]
          c_b_aux[i] = c_b[base_var[1]]
          base_aux[i] = base[base_var[1]]
        B = B_aux.copy()
        c_b = c_b_aux.copy()
        base = base_aux



        #if verbose:
          #print('\tColumnas de A en la base:')
          #for e in B:
         #   print(f'\t{list(e)}')

        #if verbose:
          #print(f'\tCoeficientes de la función objetivo de la base: {list(c_b)}')


        inversa_B = np.linalg.inv(B)                                                      # Obtenemos la matriz inversa de B.
        A = np.round(inversa_B @ A_original, 10)
        # A[A == -0.0] = 0.0
        b = np.round(inversa_B @ b_original, 10)

        z = c_b @ inversa_B @ b_original
        # print(c_b, inversa_B, A_original, c_original)
        # print(c_original)
        c = np.round((c_b @ inversa_B @ A_original) - c_original, 10)
        # c[c == -0.0] = 0.0

        #if verbose:
         # print('\tBase al final de la iteración: {'+', '.join(base)+'}')
          #print(f'\tFunción objetivo respecto a la nueva base: {list(c)}')
          #print(f'\tValor actualizado de la función objetivo: {z}')
          #it += 1

    #if svm:
      #return c

    punto = [0,0]
    for i in range(len(b)):
      if base[i] == "x1":
        punto[0] = b[i]
      if base[i] == "x2":
        punto[1] = b[i]
      #print(f'\t{var_solution[i]} = {optimal_solution[i]}')
    return punto

    #return z, b, base  

PRUEBA Simplex 

In [2]:
c = np.array([-1, -5, 0, 0])
A = np.array([[3, 1, 1, 0],[1, 4, 0, 1]])
b = np.array([9, 10])
var = np.array(['x1', 'x2', 's1', 's2'])
a = SimplexMatricial(A,b,c,var,True)
print(a)

[0, 2.5]


Algoritmo ponderaciones

In [10]:
from scipy.optimize import linprog
from sympy import symbols, Eq, solve


def calcular_soluciones_con_diferentes_ponderaciones(c, A_eq, b_eq, var):#pendiente_restriccion):
    
    

    # Inicializamos listas para almacenar las pendientes tanto de las restricciones como de las funciones objetivo
    pendientes_objetivo = []
    pendiente_solucion = []
    for i in range(len(c)):
        if c[i][1]==0:
            for i in range(len(A_eq)):
                pendiente_solucion.append([-A_eq[i][0]/A_eq[i][1],A_eq[i]])
            
    
    if pendiente_solucion == []:
        for i in range(len(c)):
            pendientes_objetivo.append(-c[i][0]/c[i][1])
        pendientes_restriccion = []
        for i in range(len(A_eq)):
            pendientes_restriccion.append([-A_eq[i][0]/A_eq[i][1],A_eq[i]])




        #En pendiente_solucion guardamos la pendiente y la recta de las restricciones que tienen pendiente entre las funciones objetivo
        #Se supone que las pendientes objetivo seran siempre 2
        for m_si in pendientes_restriccion:
            if (m_si[0]<pendientes_objetivo[1] and m_si[0]>pendientes_objetivo[0]) or (m_si[0]>pendientes_objetivo[1] and m_si[0]<pendientes_objetivo[0]):
                pendiente_solucion.append(m_si)
    #print(pendiente_solucion)


    #Para cada restricción que tenga pendiente entre las funciones objetivo, calculamos su landa1 y landa2 
    #esto lo hacemos con una fórmula que hemos deducido al generalizar para cualquier caso los pasos que seguimos en clase
    lambdas=[]
    for m in pendiente_solucion:
        A = np.array([[(c[0][0]+m[0]*c[0][1]), (c[1][0]+m[0]*c[1][1])], [1, 1]])
        b = np.array([0, 1])
        l1, l2 = np.linalg.solve(A, b)
        lambdas.append([[l1,l2],m[1]])
    #print(lambdas)



    #calculamos las landa1/landa2 de cada restricción viable. Nos guardamos también los valores x e y de esa restricción para en un futuro usar la recta.
    restricciones_sol = []
    for l in range(len(lambdas)):
        restricciones_sol.append([lambdas[l][0][0]/lambdas[l][0][1],lambdas[l][1]])
    #print(restricciones_sol)
    restricciones_sol = sorted(restricciones_sol, reverse=True)
    #print("hola"+str(restricciones_sol))
    


    #siempre tendremos un punto solución que proviene de hallar simplex con landa1 = 1 y landa2 = 0
    #Ya que cuando digamos que siendo el landa1/landa2 elegido, mayor que el l1/l2 máximo en restricciones_sol, esto se puede calcular dando total prioridad a landa 1
    #más adelante se hace lo mismo con el landa1/landa2 siendo menor que el mínimo, por tanto ahí se da prioridad a landa 2 y hallando el punto B.
    punto_A=SimplexMatricial(A_eq, b_eq,c[0],var,True)
    puntos_sol=[]
    puntos_sol.append(punto_A)
    #print(puntos_sol)


    
    #si solamente hay una restricción con pendiente entre las funciones objetivo, con los puntos A y B ya habríamos acabado.
    #si hay dos o más restricciones válidas, entonces debemos calcular los landa1 y landa2 para que su cociente sea mayor y menor a los landa1/landa2 en restricciones_sol.

    #otra forma de solucionarlo, y la que usaremos, será calcular los punto de corte entre las restricciones válidas, que al estar ya ordenadas de mayor a menor por sus landa1/landa2 sabemos que
    #nos darán los puntos solución que queremos.


    #sacar los puntos solución que se hallan haciendo intersección de las restricciones con pendiente entre las funciones objetivo (en caso de que haya 2 o más restricciones que lo cumplan)
    rectas = np.zeros((len(restricciones_sol),len(restricciones_sol[0][1])))
    #print(rectas)
    for i in range(len(restricciones_sol)):
        rectas[i] = list(restricciones_sol[i][1])
    #print(rectas)
    a_sol = []
    b_sol = []
    for i in range(len(A_eq)):
        if A_eq[i] in rectas:
            a_sol.append(list(A_eq[i]))
            b_sol.append(b_eq[i])
    #print(a_sol)
    #print(b_sol)
    
    x, y = symbols('x y')    #con esto se definen las variables para resolver el sistema con las ecuaciones de las rectas
    if len(a_sol) >= 2:
        for i in range(len(a_sol)-1):
            A1, B1, C1 = a_sol[i][0], a_sol[i][1], b_sol[i]  # Coeficientes de la primera recta
            A2, B2, C2 = a_sol[i+1][0], a_sol[i+1][1], b_sol[i+1]  # Coeficientes de la segunda recta
            # Definir las ecuaciones de las rectas en forma general
            eq1 = Eq(A1 * x + B1 * y, C1)
            eq2 = Eq(A2 * x + B2 * y, C2)

            # Resolver el sistema de ecuaciones para encontrar el punto de intersección
            intersection = solve((eq1, eq2), (x, y))

            # El punto de intersección con las coordenadas (x, y)
            puntos_sol.append(list(intersection.values()))
    #print(puntos_sol)


    #siempre tendremos un punto solución que proviene de hallar simplex con landa1 = 0 y landa2 = 1
    #Ya que cuando digamos que siendo el landa1/landa2 elegido, menor que el l1/l2 mínimo en restricciones_sol, esto se puede calcular dando total prioridad a landa 2
    punto_B=SimplexMatricial(A_eq, b_eq,c[1],var,True)
    puntos_sol.append(punto_B)
    #print(puntos_sol)

    #de esta manera en puntos_sol ya tenemos todos los puntos que serán solución, así como sus segmentos al unirlos 2 a 2.
    #lo que nos falta es hacer un print en el que digamos que dependiendo de que valores de landa1/landa2, diremos cual será el punto o segmento solución.

    for i in range(len(restricciones_sol)):
        #print(restricciones_sol)
        punto_mayor=puntos_sol[i]
        punto_menor=puntos_sol[i+1]
        segmento=puntos_sol[i:i+2]
        print("Si landa 1 / landa 2 es mayor que "+str(restricciones_sol[i][0])+" entonces el punto solución es "+str(punto_mayor))
        print("Si landa 1 / landa 2 es igual que "+str(restricciones_sol[i][0])+" entonces el segmento solución es la unión de los dos puntos "+str(segmento))
        print("Si landa 1 / landa 2 es menor que "+str(restricciones_sol[i][0])+" entonces el punto solución es "+str(punto_menor))

Ejemplo de uso

In [6]:
c = np.array([[-2, -1, 0, 0], [-1, -5, 0, 0]])  # Coeficientes de la función objetivo
A_eq = np.array([[3, 1, 1, 0], [1, 4, 0, 1]])  # Coeficientes de las restricciones de igualdad
b_eq = np.array([9, 10])  # Lados derechos de las restricciones de igualdad    
var = np.array(['x1', 'x2', 's1', 's2']) 
calcular_soluciones_con_diferentes_ponderaciones(c, A_eq, b_eq,var)

Si landa 1 / landa 2 es mayor que 0.14285714285714285 entonces el punto solución es [2.3636363636, 1.9090909091]
Si landa 1 / landa 2 es igual que 0.14285714285714285 entonces el segmento solución es la unión de los dos puntos [[2.3636363636, 1.9090909091], [0, 2.5]]
Si landa 1 / landa 2 es menor que 0.14285714285714285 entonces el punto solución es [0, 2.5]


Ejemplo de uso con dos restrcciones válidas

In [11]:
c = np.array([[-1, 0, 0, 0], [0, -1, 0, 0]])  # Coeficientes de la función objetivo
A_eq = np.array([[1, 3, 1, 0], [2, 1.5, 0, 1]])  # Coeficientes de las restricciones de igualdad
b_eq1 = np.array([90, 100])  # Lados derechos de las restricciones de igualdad    
var = np.array(['x1', 'x2', 's1', 's2']) 
calcular_soluciones_con_diferentes_ponderaciones(c, A_eq, b_eq1,var)

Si landa 1 / landa 2 es mayor que 1.3333333333333333 entonces el punto solución es [50.0, 0]
Si landa 1 / landa 2 es igual que 1.3333333333333333 entonces el segmento solución es la unión de los dos puntos [[50.0, 0], [36.6666666666667, 17.7777777777778]]
Si landa 1 / landa 2 es menor que 1.3333333333333333 entonces el punto solución es [36.6666666666667, 17.7777777777778]
Si landa 1 / landa 2 es mayor que 0.3333333333333333 entonces el punto solución es [36.6666666666667, 17.7777777777778]
Si landa 1 / landa 2 es igual que 0.3333333333333333 entonces el segmento solución es la unión de los dos puntos [[36.6666666666667, 17.7777777777778], [0, 30.0]]
Si landa 1 / landa 2 es menor que 0.3333333333333333 entonces el punto solución es [0, 30.0]


Algoritmo metas

Con el def pruebas iremos haciendo simplex con las restricciones desde la primera, añadiendo más restricciones de una en una por su orden de importancia. Se espera que el usuario entregue las restricciones en orden de importancia de mayor a menor.

In [129]:
def pruebas(A,b,c,var):
  

  numero_restricciones = 0  #para ir viendo con cuantas restricciones se ha solucionado esta iteración

  for j in range(len(A)):             #con este for creamos atest2, que serán las restricciones que se usarán para simplex en cada iteración
    A_test = A_eq[0:j+1]              #ya que hay que adaptarlo en cada iteración a las s1,s2,s3... que vaya necesitando dependiendo del numero de restricciones que haya
    atest2 = np.zeros((len(A_test),j+3))
    for i in range(len(A_test)):
      atest2[i] = list(A_test[i][0:j+3])
    #print(atest2)

    nueva_objetivo = atest2[-1]
    nueva_objetivo2 = np.concatenate((nueva_objetivo[0:2],np.zeros(j)))   #en caso de que en cada iteeración, no sean compatibles las restricciones
                                                                          #creamos nuev_objetivo para que la última restricción, que es la que da error, 
                                                                          #sea la nueva función objetivo
    
    b_test = b[0:j+1]    #adaptamos b, c y var dependiendo del numero de restricciones en cada iteracion
    c_test=c[0:j+3]
    var_test = var[0:j+3]
    #print(b_test)
    #print(c_test)
    #print(var_test)
    numero_restricciones += 1
    print('Simplex con %d restricciones:' %numero_restricciones)
    try:
      punto = SimplexMatricial(atest2,b_test,c_test,var_test,True) 
             
      print(punto)     #el punto solución cada vez que vamos metiendo una restriccion
    except:
      j = j-1
      break

  return j, nueva_objetivo2 #hacemos el simplex y si hay un error, es decir, no hay región factible nos devuelve el valor de j que es la posición de la meta problemática, que modificaremos en def metas().

Función principal del algoritmo por metas, el usuario entrega un a, b, c y var de la misma manera que en la función SimplexMatricial para que esta función pueda funcionar.

In [158]:
def metas(A, b, c, var):

  #Comprobamos si hemos hallado una restricción que no sea compatible
  j,nueva_objetivo = pruebas(A,b,c,var)  #cojo la posición de la que da error (en caso de que haya) y pasará a ser la funcion objetivo
  #print(j)
  if j == len(A)-1:     #si no hay ninguna restricción que de problemas, hace simplex tal y como vienen los datos dados
      #print(A,b,c,var)
      punto = SimplexMatricial(A,b,c,var,True)
      print('El simplex se puede hacer con la/s restricción/es y su proceso y resultado es: '+str(punto))
      

  # Encontramos la meta problemática
  else:      #si hay una restricción no factible, esta pasa a ser la objetivo
      print('No se puede aplicar simplex pues la región factible queda nula')
      # Cambiamos la función objetivo por la restricción problemática
      c = nueva_objetivo*-1 # minimizar la restricción problemática para que esté dentro de la región factible.
      A_test = A_eq[0:j+1]   #el A se adapta a las restricciones que tenemos, es j+1 porque si por ejemplo la segunda restriccion falla,
                             #j será 0, y de A debo coger solo la primera restricción
      atest2 = np.zeros((len(A_test),j+3))    
      for i in range(len(A_test)):
        atest2[i] = list(A_test[i][0:j+3])    #creamos atest2 con las restricciones necesarios para el caso 
      b = b[0:j]   #se seleccionan las b necesarias
      
      punto = SimplexMatricial(atest2,b,c,var,True)
      print('Hacemos símplex para minimizar la diferencia entre la restricción problemática y el punto de compromiso. '+str(punto))
      

Ejemplo de uso con restricciones con región factible

In [159]:
c = np.array([-1, -1, 0, 0, 0])  # Coeficientes de la función objetivo
A_eq = np.array([[4, 2, 1, 0, 0], [2, 3, 0, 1, 0], [6, -1, 0, 0, 1]])  # Coeficientes de las restricciones de igualdad
b_eq1 = np.array([8, 10, 9])  # Lados derechos de las restricciones de igualdad    
var = np.array(['x1', 'x2', 's1', 's2','s3']) 
metas(A_eq,b_eq1,c,var)

Simplex con 1 restricciones:
[0, 4.0]
Simplex con 2 restricciones:
[0.5, 3.0]
Simplex con 3 restricciones:
[0.5, 3.0]
El simplex se puede hacer con la/s restricción/es y su proceso y resultado es: [0.5, 3.0]


Ejemplo de uso con restricciones con región no factible.

In [160]:
c = np.array([-1, -1, 0, 0, 0, 0])  # Coeficientes de la función objetivo
A_eq = np.array([[4, 2, 1, 0, 0, 0], [2, 3, 0, 1, 0, 0], [6, -1, 0, 0, 1, 0],[5, -3, 0, 0, 0, 1]])  # Coeficientes de las restricciones de igualdad
b_eq1 = np.array([8, 10, 2, -13])  # Lados derechos de las restricciones de igualdad    
var = np.array(['x1', 'x2', 's1', 's2','s3', 's4']) 
metas(A_eq,b_eq1,c,var)

Simplex con 1 restricciones:
[0, 4.0]
Simplex con 2 restricciones:
[0.5, 3.0]
Simplex con 3 restricciones:
[0.5, 3.0]
Simplex con 4 restricciones:
[0.5, 3.0]
El simplex se puede hacer con la/s restricción/es y su proceso y resultado es: [0.5, 3.0]


El problema es que la función de SimplexMatricial que nos han compartido no da error cuando la región no es factible, por tanto no entra en nuestro except. Entendemos que programar simplex excede los contenidos de nuestro trabajo, si se le entregara a nuestro algoritmo de metas un algoritmo de simplex que cuando la región de las restricciones no sea factible, este levante un error, en ese caso nuestro algoritmo daría la respuesta apropiada.

In [139]:
SimplexMatricial(A_eq,b_eq1,c,var)
#aqui vemos que devuelve un resultado cuando la región no es factible.

[0.5, 3.0]

Comprobamos que linprog tampoco levanta un error y por tanto no puede ser utilizado, lo que hace es devolver ya un resultado aproximado, haciendo nuestro trabajo al solucionarlo por metas, pero no permitiendo que de error y que podamos aplicar nuestro algoritmo

In [156]:
from scipy.optimize import linprog

c = [1, 1]
a = [[4, 2], [2, 3], [6, -1], [5, -3]]
b = [8, 10, 2, -13]
try:
    res = linprog(c, A_ub=a, b_ub=b, method="simplex")
    print(res.x)
    print(res.status)
except:
    print(1)

[0.         3.33333333]
2


Entendemos que la única manera de ver que las regiones no son factibles, es utilizando un simplex que levante un error cuando esto suceda, ya que de otra manera no podemos ver que al añadir una restricción esta no sea factible. hemos visto que linprog devuelve un status en el que si este es 2, significa que la solución no es factible. Si es necesario que esto se devuelva bien para que la práctica sea correcta, podemos adaptar el algoritmo a que se use linprog, ya que ahora mismo linprog no recibe las variables de la misma forma en la que las tenemos. Pero no hemos hemos hecho esto ya que anteriormente habíamos mencionado que linprog no nos devuelve correctamente los resultados de simplex.